In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

**# Set dataset path** **bold text**

In [3]:
data_dir = "./samples"  # your actual dataset folder path


# Load dataset with automatic label inference from folder names **bold text**

In [4]:

batch_size = 32
img_height = 224
img_width = 224

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
# Get class names
class_names = train_dataset.class_names
num_classes = len(class_names)

Found 1611 files belonging to 26 classes.
Using 1289 files for training.
Found 1611 files belonging to 26 classes.
Using 322 files for validation.


In [5]:
# Improve performance with prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

In [6]:
# Load ResNet50 as base model
base_model = tf.keras.applications.ResNet50(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model for transfer learning

# Build model
model = models.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip (RandomFlip)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation (RandomRotation)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom (RandomZoom)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
# Train the model
epochs = 30  # You can increase this
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)


Epoch 1/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 192ms/step - accuracy: 0.4392 - loss: 1.6301 - val_accuracy: 0.3602 - val_loss: 1.6862
Epoch 2/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.4733 - loss: 1.5733 - val_accuracy: 0.4068 - val_loss: 1.5992
Epoch 3/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 11s 221ms/step - accuracy: 0.4768 - loss: 1.4754 - val_accuracy: 0.4441 - val_loss: 1.5041
Epoch 4/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 10s 209ms/step - accuracy: 0.4735 - loss: 1.4879 - val_accuracy: 0.4037 - val_loss: 1.5039
Epoch 5/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 10s 209ms/step - accuracy: 0.4688 - loss: 1.4528 - val_accuracy: 0.4224 - val_loss: 1.4135
Epoch 6/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 9s 223ms/step - accuracy: 0.5230 - loss: 1.3749 - val_accuracy: 0.4441 - val_loss: 1.4029
Epoch 7/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 14s 305ms/step - accuracy: 0.5308 - loss: 1.3154 - val_accuracy: 0.4317 - val_loss: 1.4091
Epoch 8/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 16s 185ms/step - accuracy: 0.5569 - loss: 1.3089 - val_accurac

# Save the **model**

In [10]:

model.save("gear_classifier_model.keras")
print("✅ Model saved as 'gear_classifier_model.keras'")


✅ Model saved as 'gear_classifier_model.keras'


### **bellow cell is to unzip the zip file of the dataset**

In [11]:
import zipfile
import os

zip_path = 'samples.zip'  # your uploaded file name
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./')  # extract to a folder named 'data'
